In [1]:
! pip install -U opencv-contrib-python
! git clone https://github.com/naver/r2d2.git
# ! pip install opencv-python-headless==4.1.2.30

Defaulting to user installation because normal site-packages is not writeable
fatal: destination path 'r2d2' already exists and is not an empty directory.


In [2]:
! pwd

/home/aiarhipov/OCR/course_ocr/task1


In [3]:
from pathlib import Path
from course_ocr_t1.data import MidvPackage
from tqdm import tqdm
from matplotlib import pyplot as plt
import numpy as np
from tqdm.notebook import tqdm
import cv2


In [4]:
DATASET_PATH = Path('/home/aiarhipov/OCR/midv500_compressed')
assert DATASET_PATH.exists(), DATASET_PATH.absolute()

In [5]:
! cd midv500_compressed ; cd 50_xpo_id ; cd images ; ls 

/bin/bash: line 0: cd: midv500_compressed: No such file or directory
/bin/bash: line 0: cd: 50_xpo_id: No such file or directory
/bin/bash: line 0: cd: images: No such file or directory
README.md	df-Copy1.xlsx  gt.json	       r2d2
baseline.ipynb	df.xlsx        pred-96,7.json  r2d2.ipynb
course_ocr_t1	df_99%.xlsx    pred.json       setup.py


всего 15050 фоток

In [6]:
# Собираем список пакетов (MidvPackage) 
data_packs = MidvPackage.read_midv500_dataset(DATASET_PATH)
len(data_packs), type(data_packs[0])

(50, course_ocr_t1.data.MidvPackage)

In [7]:
import sys
r2d2_path = '/home/aiarhipov/OCR/course_ocr/task1/r2d2/'
if r2d2_path not in sys.path:
    sys.path.append(r2d2_path)


In [8]:
! cd r2d2

In [9]:
import os, pdb
from PIL import Image
import numpy as np
import torch

from r2d2.tools import common
from r2d2.nets.patchnet import *
import torchvision.transforms as tvf
RGB_mean = [0.485, 0.456, 0.406]
RGB_std  = [0.229, 0.224, 0.225]

norm_RGB = tvf.Compose([tvf.ToTensor(), tvf.Normalize(mean=RGB_mean, std=RGB_std)])

In [10]:
def load_network(model_fn): 
    checkpoint = torch.load(model_fn)
    net = eval(checkpoint['net'])
    nb_of_weights = common.model_size(net)

    weights = checkpoint['state_dict']
    net.load_state_dict({k.replace('module.',''):v for k,v in weights.items()})
    return net.eval()
class NonMaxSuppression (torch.nn.Module):
    def __init__(self, rel_thr=0.7, rep_thr=0.7):
        nn.Module.__init__(self)
        self.max_filter = torch.nn.MaxPool2d(kernel_size=3, stride=1, padding=1)
        self.rel_thr = rel_thr
        self.rep_thr = rep_thr
    
    def forward(self, reliability, repeatability, **kw):
        assert len(reliability) == len(repeatability) == 1
        reliability, repeatability = reliability[0], repeatability[0]

        maxima = (repeatability == self.max_filter(repeatability))

        maxima *= (repeatability >= self.rep_thr)
        maxima *= (reliability   >= self.rel_thr)

        return maxima.nonzero().t()[2:4]
def extract_multiscale( net, img, detector, scale_f=2**0.25, 
                        min_scale=0.0, max_scale=1, 
                        min_size=256, max_size=1024, 
                        verbose=False):
    old_bm = torch.backends.cudnn.benchmark 
    torch.backends.cudnn.benchmark = False # speedup
    
    B, three, H, W = img.shape
    assert B == 1 and three == 3, "should be a batch with a single RGB image"
    
    assert max_scale <= 1
    s = 1.0 # current scale factor
    
    X,Y,S,C,Q,D = [],[],[],[],[],[]
    while  s+0.001 >= max(min_scale, min_size / max(H,W)):
        if s-0.001 <= min(max_scale, max_size / max(H,W)):
            nh, nw = img.shape[2:]
            if verbose: print(f"extracting at scale x{s:.02f} = {nw:4d}x{nh:3d}")
            with torch.no_grad():
                res = net(imgs=[img])
                
 
            descriptors = res['descriptors'][0]
            reliability = res['reliability'][0]
            repeatability = res['repeatability'][0]


            y,x = detector(**res) # nms
            c = reliability[0,0,y,x]
            q = repeatability[0,0,y,x]
            d = descriptors[0,:,y,x].t()
            n = d.shape[0]

            X.append(x.float() * W/nw)
            Y.append(y.float() * H/nh)
            S.append((32/s) * torch.ones(n, dtype=torch.float32, device=d.device))
            C.append(c)
            Q.append(q)
            D.append(d)
        s /= scale_f

        nh, nw = round(H*s), round(W*s)
        img = F.interpolate(img, (nh,nw), mode='bilinear', align_corners=False)

    torch.backends.cudnn.benchmark = old_bm

    Y = torch.cat(Y)
    X = torch.cat(X)
    S = torch.cat(S) # scale
    scores = torch.cat(C) * torch.cat(Q) # scores = reliability * repeatability
    XYS = torch.stack([X,Y,S], dim=-1)
    D = torch.cat(D)
    return XYS, D, scores

In [11]:
net = load_network('/home/aiarhipov/OCR/course_ocr/task1/r2d2/models/r2d2.pt')
iscuda = common.torch_set_gpu(1)
if iscuda: net = net.cuda()

def R2D2_hard(img_path, top_k=5000):

    detector = NonMaxSuppression(
        rel_thr = 0.7, 
        rep_thr = 0.7)

    img = Image.open(img_path).convert('RGB')
    W, H = img.size
    img = norm_RGB(img)[None] 
    if iscuda: img = img.cuda()
    
    xys, desc, scores = extract_multiscale(net, img, detector,
        scale_f   = 2**0.25, 
        min_scale = 0.1, 
        max_scale = 1,
        min_size  = 0, 
        max_size  = 9999, 
        verbose = False)

    xys = xys.cpu().numpy()
    desc = desc.cpu().numpy()
    scores = scores.cpu().numpy()
    idxs = scores.argsort()[-top_k or None:]

    imsize = (W,H),
    keypoints = xys[idxs], 
    descriptors = desc[idxs], 
    scores = scores[idxs]

    return keypoints, descriptors, scores

Launching on GPUs 1


In [31]:
from shapely import geometry
import gc

results_dict = dict()

for j in tqdm(range(len(data_packs)), desc='Directory', leave=True, position=0):
    dp = data_packs[j]
    
    torch.cuda.empty_cache()
    
    kp, ft, sc = R2D2_hard(str(dp.template_item.img_path))


    orig_kp = kp[0]
    orig_ft = ft[0]

    
    for i in tqdm(range(len(dp)), desc='Files', leave=False, position=1):
        
        if dp[i].is_test_split():
            try:
                kp, ft, sc = R2D2_hard(str(dp[i].img_path))

                real_kp = kp[0]
                real_ft = ft[0]


                bf = cv2.BFMatcher()
                matches = bf.knnMatch(real_ft, orig_ft, k=2)

                best = []
                for match1,match2 in matches:
                    if match1.distance <  0.9 * match2.distance:
                        best.append(match1)

                if len(best) < 4:
                    best=[]
                    for match1,match2 in matches:
                        best.append(match1)

                real_points = np.zeros((len(best), 2), dtype="float")
                orig_points = np.zeros((len(best), 2), dtype="float")

                for (k, m) in enumerate(best):
                    real_points[k] = real_kp[m.queryIdx][:2]
                    orig_points[k] = orig_kp[m.trainIdx][:2]

                (homography, mask) = cv2.findHomography(orig_points, real_points, method=cv2.RANSAC)

                real_array = np.array(dp[i].image)
                orig_array = np.array(dp.template_item.image)
                real_h = real_array.shape[0]
                real_w = real_array.shape[1]
                orig_h = float(orig_array.shape[0])
                orig_w = float(orig_array.shape[1])
                                
                points = [
                    [0,      0     ],
                    [orig_w, 0     ],
                    [orig_w, orig_h],
                    [0,      orig_h],
                ]
                
                crop = list()
                for point in points:
                    vec = homography @ np.array([point[0], point[1], 1])
                    crop.append([vec[0]/vec[-1], vec[1]/vec[-1]])

                pred_quad = np.array(crop) / np.array([[real_w, real_h]])

                del kp, ft, sc, real_kp, real_ft
                gc.collect()
                
                try:
                    _ = geometry.Polygon(pred_quad)
                    results_dict[dp[i].unique_key] = pred_quad

                except Exception as exc1:
                    print(exc1)


            except Exception as exc2:
                print(exc2)
    del orig_kp, orig_ft
    gc.collect()

matmul: Input operand 0 does not have enough dimensions (has 0, gufunc core with signature (n?,k),(k,m?)->(n?,m?) requires 1)
matmul: Input operand 0 does not have enough dimensions (has 0, gufunc core with signature (n?,k),(k,m?)->(n?,m?) requires 1)
matmul: Input operand 0 does not have enough dimensions (has 0, gufunc core with signature (n?,k),(k,m?)->(n?,m?) requires 1)


matmul: Input operand 0 does not have enough dimensions (has 0, gufunc core with signature (n?,k),(k,m?)->(n?,m?) requires 1)
matmul: Input operand 0 does not have enough dimensions (has 0, gufunc core with signature (n?,k),(k,m?)->(n?,m?) requires 1)


In [36]:
from course_ocr_t1.metrics import dump_results_dict, measure_crop_accuracy

In [37]:
dump_results_dict(results_dict, Path() / 'pred.json')

In [38]:
acc = measure_crop_accuracy(
    Path() / 'pred.json',
    Path() / 'gt.json'
)

In [39]:
print("Точность кропа: {:1.4f}".format(acc))

Точность кропа: 0.9699
